# Experiment: Staer (2017) 近似再現 (Phase B)

本ノートブックは `prices.csv` のみを使い、論文の分析フレーム（パネル回帰・VAR・週次ETF vs MF）を近似再現する。

## 近似再現の前提

- 厳密再現ではない（`Shrout`, `NAV`, ICI週次MFフローが未提供）。
- 代替定義:
  - `ret_{i,t} = AdjClose_{i,t}/AdjClose_{i,t-1} - 1`
  - `flow_proxy_{i,t} = diff(log(Close_{i,t} * Volume_{i,t}))`
  - `premium_proxy_{i,t} = Close_{i,t}/Open_{i,t} - 1`
  - `market_ret_t = mean_i(ret_{i,t})`, `flow_aggr_t = mean_i(flow_proxy_{i,t})`
- 出力先:
  - `/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/*.csv`
  - `/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_evidence_summary_2026-02-23.md`

In [1]:
from pathlib import Path
import shutil
import warnings

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

warnings.filterwarnings('ignore')

BASE = Path('/Users/kencharoff/workspace/projects/momentum/p04_fund_flows')
DATA_PATH = BASE / 'data/prices.csv'
MF_PATH = BASE / 'data/raw/mf_weekly_flows.csv'

OUTPUT_DIR = BASE / 'outputs'
RESULT_DIR = OUTPUT_DIR / 'phaseB_results'
EVIDENCE_PATH = OUTPUT_DIR / 'phaseB_evidence_summary_2026-02-23.md'
RESULT_DIR.mkdir(parents=True, exist_ok=True)


def backup_if_exists(path: Path) -> Path | None:
    if not path.exists():
        return None
    ts = pd.Timestamp.now(tz='Asia/Tokyo').strftime('%Y%m%d%H%M%S')
    backup = path.with_name(f'{path.name}.bak.{ts}')
    shutil.copy2(path, backup)
    return backup

In [2]:
raw = pd.read_csv(DATA_PATH, header=[0, 1], index_col=0)
raw.index = pd.to_datetime(raw.index)
raw = raw.sort_index()

required_level0 = {'Adj Close', 'Close', 'Open', 'Volume'}
level0 = set(raw.columns.get_level_values(0))
missing = required_level0 - level0
if missing:
    raise ValueError(f'Missing required top-level columns: {sorted(missing)}')
if raw.isna().any().any():
    raise ValueError('prices.csv includes NaN values; this notebook expects no missing values.')
if not raw.index.is_monotonic_increasing:
    raise ValueError('Date index must be sorted ascending.')

adj_close = raw['Adj Close'].astype(float)
close = raw['Close'].astype(float)
open_ = raw['Open'].astype(float)
volume = raw['Volume'].astype(float)

ret = adj_close.pct_change()
dollar_volume = close * volume
flow_proxy = np.log(dollar_volume).diff()
premium_proxy = close / open_ - 1.0

market_ret = ret.mean(axis=1).rename('market_ret')
flow_aggr = flow_proxy.mean(axis=1).rename('flow_aggr')

print('date_range:', raw.index.min().date(), 'to', raw.index.max().date())
print('tickers:', sorted(adj_close.columns.tolist()))
print('rows, cols:', raw.shape)

date_range: 2011-01-03 to 2026-02-20
tickers: ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
rows, cols: (3806, 54)


In [3]:
ret_long = ret.stack().rename('ret')
flow_long = flow_proxy.stack().rename('flow_proxy')
premium_long = premium_proxy.stack().rename('premium_proxy')

panel = pd.concat([ret_long, flow_long, premium_long], axis=1).reset_index()
panel.columns = ['date', 'ticker', 'ret', 'flow_proxy', 'premium_proxy']

for lag in range(1, 6):
    panel[f'market_ret_lag{lag}'] = panel['date'].map(market_ret.shift(lag))
for lag in range(0, 6):
    panel[f'flow_proxy_lag{lag}'] = panel.groupby('ticker')['flow_proxy'].shift(lag)
for lag in range(0, 2):
    panel[f'premium_proxy_lag{lag}'] = panel.groupby('ticker')['premium_proxy'].shift(lag)

panel['ret_lead1'] = panel.groupby('ticker')['ret'].shift(-1)

subperiods = {
    '2011_2026': ('2011-01-03', '2026-02-20'),
    '2018_2026': ('2018-01-01', '2026-02-20'),
    '2020_2026': ('2020-01-01', '2026-02-20'),
}

feature_cols = [
    *[f'market_ret_lag{lag}' for lag in range(1, 6)],
    *[f'flow_proxy_lag{lag}' for lag in range(0, 6)],
    *[f'premium_proxy_lag{lag}' for lag in range(0, 2)],
]

panel.head()

date ticker  ret  flow_proxy  premium_proxy  market_ret_lag1  \
0 2011-01-03    XLB  NaN         NaN      -0.002576              NaN   
1 2011-01-03    XLE  NaN         NaN      -0.002032              NaN   
2 2011-01-03    XLF  NaN         NaN       0.008040              NaN   
3 2011-01-03    XLI  NaN         NaN       0.001997              NaN   
4 2011-01-03    XLK  NaN         NaN       0.003546              NaN   

   market_ret_lag2  market_ret_lag3  market_ret_lag4  market_ret_lag5  \
0              NaN              NaN              NaN              NaN   
1              NaN              NaN              NaN              NaN   
2              NaN              NaN              NaN              NaN   
3              NaN              NaN              NaN              NaN   
4              NaN              NaN              NaN              NaN   

   flow_proxy_lag0  flow_proxy_lag1  flow_proxy_lag2  flow_proxy_lag3  \
0              NaN              NaN              NaN              NaN   
1              NaN              NaN              NaN              NaN   
2              NaN              NaN              NaN              NaN   
3              NaN              NaN              NaN              NaN   
4              NaN              NaN              NaN              NaN   

   flow_proxy_lag4  flow_proxy_lag5  premium_proxy_lag0  premium_proxy_lag1  \
0              NaN              NaN           -0.002576                 NaN   
1              NaN              NaN           -0.002032                 NaN   
2              NaN              NaN            0.008040                 NaN   
3              NaN              NaN            0.001997                 NaN   
4              NaN              NaN            0.003546                 NaN   

   ret_lead1  
0  -0.006715  
1  -0.008580  
2  -0.001227  
3  -0.001424  
4   0.003926

In [4]:
def run_panel_regression(panel_df: pd.DataFrame, sub_name: str, start: str, end: str):
    use = panel_df[(panel_df['date'] >= pd.Timestamp(start)) & (panel_df['date'] <= pd.Timestamp(end))].copy()
    use = use.dropna(subset=['ret', *feature_cols])

    dummies = pd.get_dummies(use['ticker'], prefix='ticker', drop_first=True, dtype=float)
    X = pd.concat([use[feature_cols], dummies], axis=1)
    X = sm.add_constant(X)
    y = use['ret']

    fit = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})

    rows = []
    for var in ['const', *feature_cols]:
        rows.append({
            'subperiod': sub_name,
            'variable': var,
            'coef': float(fit.params.get(var, np.nan)),
            'tvalue': float(fit.tvalues.get(var, np.nan)),
            'pvalue': float(fit.pvalues.get(var, np.nan)),
            'nobs': int(fit.nobs),
            'r2': float(fit.rsquared),
        })
    return pd.DataFrame(rows), fit


def run_var_analysis(market_ret_s: pd.Series, flow_aggr_s: pd.Series, sub_name: str, start: str, end: str):
    var_df = pd.concat([market_ret_s, flow_aggr_s], axis=1).loc[start:end].dropna()
    fit = VAR(var_df).fit(4)

    coef = fit.params.stack().rename('coef').reset_index()
    coef.columns = ['regressor', 'equation', 'coef']
    tvals = fit.tvalues.stack().rename('tvalue').reset_index(drop=True)
    pvals = fit.pvalues.stack().rename('pvalue').reset_index(drop=True)

    out = coef.copy()
    out['tvalue'] = tvals
    out['pvalue'] = pvals
    out.insert(0, 'subperiod', sub_name)
    out['nobs'] = int(fit.nobs)

    irf = fit.irf(20)
    response = irf.irfs[:, 0, 1]  # response=market_ret, impulse=flow_aggr
    cirf = np.cumsum(response)
    cirf_df = pd.DataFrame({
        'subperiod': sub_name,
        'horizon': np.arange(len(response)),
        'irf_market_to_flow': response,
        'cirf_market_to_flow': cirf,
    })
    return out, cirf_df, fit


def run_optional_weekly_section(mf_path: Path, market_ret_s: pd.Series, dollar_volume_df: pd.DataFrame):
    output_path = RESULT_DIR / 'weekly_lead_coefficients.csv'
    market_ret_daily = market_ret_s.dropna()

    if not mf_path.exists():
        weekly_out = pd.DataFrame([{'status': 'skip', 'reason': 'missing mf_weekly_flows.csv'}])
        weekly_out.to_csv(output_path, index=False)
        return weekly_out, 'skipped_missing_mf'

    mf = pd.read_csv(mf_path)
    required_cols = {'week_end', 'flow_w_mf_bil'}
    if not required_cols.issubset(set(mf.columns)):
        raise ValueError(f'mf_weekly_flows.csv must include columns: {sorted(required_cols)}')

    mf['week_end'] = pd.to_datetime(mf['week_end'])
    mf = mf.sort_values('week_end')

    dollar_volume_agg_bil = (dollar_volume_df.sum(axis=1) / 1e9).rename('dollar_vol_bil')
    etf_weekly = dollar_volume_agg_bil.resample('W-FRI').last().diff().rename('flow_w_etf_bil')

    rows = []
    trading_index = market_ret_daily.index
    weekly_index = etf_weekly.dropna().index

    for dt in weekly_index:
        pos = trading_index.searchsorted(dt, side='right') - 1
        if pos < 0 or pos + 7 >= len(trading_index):
            continue
        row = {'week_end': dt, 'flow_w_etf_bil': float(etf_weekly.loc[dt])}
        row['ret_week_t'] = float(market_ret_daily.iloc[max(0, pos - 4):pos + 1].sum())
        for k in range(0, 8):
            row[f'ret_t_plus_{k}'] = float(market_ret_daily.iloc[pos + k])
        rows.append(row)

    wk = pd.DataFrame(rows)
    wk = wk.merge(mf[['week_end', 'flow_w_mf_bil']], on='week_end', how='inner')

    weekly_results = []
    for k in range(1, 8):
        y_col = f'ret_t_plus_{k}'
        ctrl_cols = [f'ret_t_plus_{j}' for j in range(0, k)]
        x_cols = ctrl_cols + ['flow_w_etf_bil', 'flow_w_mf_bil', 'ret_week_t']
        reg = wk.dropna(subset=[y_col, *x_cols]).copy()
        if reg.empty:
            weekly_results.append({'k': k, 'status': 'skip_empty'})
            continue

        X = sm.add_constant(reg[x_cols])
        y = reg[y_col]
        fit = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})

        weekly_results.append({
            'k': k,
            'status': 'ok',
            'nobs': int(fit.nobs),
            'coef_flow_w_etf_bil': float(fit.params.get('flow_w_etf_bil', np.nan)),
            't_flow_w_etf_bil': float(fit.tvalues.get('flow_w_etf_bil', np.nan)),
            'p_flow_w_etf_bil': float(fit.pvalues.get('flow_w_etf_bil', np.nan)),
            'coef_flow_w_mf_bil': float(fit.params.get('flow_w_mf_bil', np.nan)),
            't_flow_w_mf_bil': float(fit.tvalues.get('flow_w_mf_bil', np.nan)),
            'p_flow_w_mf_bil': float(fit.pvalues.get('flow_w_mf_bil', np.nan)),
            'r2': float(fit.rsquared),
        })

    weekly_out = pd.DataFrame(weekly_results)
    weekly_out.to_csv(output_path, index=False)
    return weekly_out, 'executed'

In [5]:
panel_rows = []
panel_fits = {}
var_rows = []
var_fits = {}
cirf_rows = []

for sub_name, (start, end) in subperiods.items():
    p_df, p_fit = run_panel_regression(panel, sub_name, start, end)
    v_df, c_df, v_fit = run_var_analysis(market_ret, flow_aggr, sub_name, start, end)

    panel_rows.append(p_df)
    var_rows.append(v_df)
    cirf_rows.append(c_df)
    panel_fits[sub_name] = p_fit
    var_fits[sub_name] = v_fit

panel_out = pd.concat(panel_rows, ignore_index=True)
var_out = pd.concat(var_rows, ignore_index=True)
cirf_out = pd.concat(cirf_rows, ignore_index=True)

panel_path = RESULT_DIR / 'panel_coefficients.csv'
var_path = RESULT_DIR / 'var_coefficients.csv'
cirf_path = RESULT_DIR / 'cirf_table.csv'

panel_out.to_csv(panel_path, index=False)
var_out.to_csv(var_path, index=False)
cirf_out.to_csv(cirf_path, index=False)

weekly_out, weekly_status = run_optional_weekly_section(MF_PATH, market_ret, dollar_volume)

print('saved:', panel_path)
print('saved:', var_path)
print('saved:', cirf_path)
print('saved:', RESULT_DIR / 'weekly_lead_coefficients.csv')
print('weekly_status:', weekly_status)

panel_out.head(10)

saved: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/panel_coefficients.csv
saved: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/var_coefficients.csv
saved: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/cirf_table.csv
saved: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/weekly_lead_coefficients.csv
weekly_status: skipped_missing_mf


subperiod         variable      coef    tvalue        pvalue   nobs  \
0  2011_2026            const  0.000286  2.028978  4.246053e-02  34200   
1  2011_2026  market_ret_lag1 -0.087504 -2.865375  4.165158e-03  34200   
2  2011_2026  market_ret_lag2  0.040243  1.437217  1.506562e-01  34200   
3  2011_2026  market_ret_lag3 -0.000227 -0.009549  9.923812e-01  34200   
4  2011_2026  market_ret_lag4 -0.042715 -2.077594  3.774675e-02  34200   
5  2011_2026  market_ret_lag5  0.024644  1.109871  2.670545e-01  34200   
6  2011_2026  flow_proxy_lag0 -0.001586 -6.034529  1.594270e-09  34200   
7  2011_2026  flow_proxy_lag1 -0.001276 -4.620940  3.820050e-06  34200   
8  2011_2026  flow_proxy_lag2 -0.000655 -2.054332  3.994355e-02  34200   
9  2011_2026  flow_proxy_lag3 -0.000160 -0.472657  6.364580e-01  34200   

         r2  
0  0.629857  
1  0.629857  
2  0.629857  
3  0.629857  
4  0.629857  
5  0.629857  
6  0.629857  
7  0.629857  
8  0.629857  
9  0.629857

In [6]:
# Leakage防止チェック: ret_{t+1} を flow_proxy_t だけで説明する補助回帰
next_date = panel.groupby('ticker')['date'].shift(-1)
valid = panel['ret_lead1'].notna()
assert (next_date[valid] > panel.loc[valid, 'date']).all(), 'ret_lead1 alignment must be strictly future.'

leak_df = panel.dropna(subset=['ret_lead1', 'flow_proxy_lag0']).copy()
X_leak = sm.add_constant(leak_df[['flow_proxy_lag0']])
leak_fit = sm.OLS(leak_df['ret_lead1'], X_leak).fit(cov_type='HAC', cov_kwds={'maxlags': 5})

pd.DataFrame({
    'coef': [leak_fit.params['flow_proxy_lag0']],
    'tvalue': [leak_fit.tvalues['flow_proxy_lag0']],
    'pvalue': [leak_fit.pvalues['flow_proxy_lag0']],
    'nobs': [int(leak_fit.nobs)],
}, index=['ret_(t+1) ~ flow_proxy_t'])

coef    tvalue    pvalue   nobs
ret_(t+1) ~ flow_proxy_t -0.00062 -2.396733  0.016542  34236

In [7]:

backup_path = backup_if_exists(EVIDENCE_PATH)

flow_sign_rows = []
for sub_name in subperiods:
    psub = panel_out[panel_out['subperiod'] == sub_name].set_index('variable')
    flow_sign_rows.append({
        'subperiod': sub_name,
        'flow_t_coef': float(psub.loc['flow_proxy_lag0', 'coef']),
        'flow_t_pvalue': float(psub.loc['flow_proxy_lag0', 'pvalue']),
        'flow_t_minus_2_coef': float(psub.loc['flow_proxy_lag2', 'coef']),
        'flow_t_minus_2_pvalue': float(psub.loc['flow_proxy_lag2', 'pvalue']),
    })
flow_sign_df = pd.DataFrame(flow_sign_rows)

var_sign_rows = []
for sub_name in subperiods:
    mask = (
        (var_out['subperiod'] == sub_name)
        & (var_out['equation'] == 'market_ret')
        & (var_out['regressor'] == 'L2.flow_aggr')
    )
    row = var_out.loc[mask]
    if row.empty:
        continue
    rr = row.iloc[0]
    var_sign_rows.append({
        'subperiod': sub_name,
        'coef_L2_flow_aggr_to_market_ret': float(rr['coef']),
        'pvalue': float(rr['pvalue']),
    })
var_sign_df = pd.DataFrame(var_sign_rows)

lines = []
lines.append('# Phase B Evidence Summary (2026-02-23)')
lines.append('')
lines.append(f'- 生成日時: {pd.Timestamp.now(tz="Asia/Tokyo").strftime("%Y-%m-%d %H:%M:%S %Z")}')
lines.append(f'- 入力データ: `{DATA_PATH}`')
lines.append(f'- 任意MFデータ: `{MF_PATH}`')
lines.append(f'- MFセクション状態: `{weekly_status}`')
if backup_path is not None:
    lines.append(f'- 既存証跡バックアップ: `{backup_path}`')
lines.append('')
lines.append('## 近似定義')
lines.append('- `ret_{i,t} = AdjClose_{i,t} / AdjClose_{i,t-1} - 1`')
lines.append('- `flow_proxy_{i,t} = diff(log(Close_{i,t} * Volume_{i,t}))`')
lines.append('- `premium_proxy_{i,t} = Close_{i,t} / Open_{i,t} - 1`')
lines.append('- `market_ret_t = mean_i(ret_{i,t})`, `flow_aggr_t = mean_i(flow_proxy_{i,t})`')
lines.append('')
lines.append('## パネル回帰主要符号（近似）')
lines.append(flow_sign_df.to_markdown(index=False))
lines.append('')
lines.append('## VAR主要係数（market_ret方程式のL2.flow_aggr）')
if not var_sign_df.empty:
    lines.append(var_sign_df.to_markdown(index=False))
else:
    lines.append('- 該当係数を抽出できませんでした。')
lines.append('')
lines.append('## リーク防止検証')
lines.append(f'- `ret_(t+1) ~ flow_proxy_t` 補助回帰: coef={leak_fit.params["flow_proxy_lag0"]:.6g}, p={leak_fit.pvalues["flow_proxy_lag0"]:.6g}')
lines.append('- 特徴量はすべて `t` 時点以前のラグのみで構築し、目的変数は `t+1` を別列化して検証。')
lines.append('')
lines.append('## 未再現項目')
lines.append('- 原著のShrout/NAVベースflow、Premium、ICI週次MFフローが未提供のため厳密再現ではない。')
lines.append('- 本成果は `prices.csv` に基づく近似再現であり、論文の絶対値一致は目的外。')
lines.append('')
lines.append('## 出力ファイル')
lines.append(f'- `{RESULT_DIR / "panel_coefficients.csv"}`')
lines.append(f'- `{RESULT_DIR / "var_coefficients.csv"}`')
lines.append(f'- `{RESULT_DIR / "cirf_table.csv"}`')
lines.append(f'- `{RESULT_DIR / "weekly_lead_coefficients.csv"}`')
lines.append(f'- `{EVIDENCE_PATH}`')

EVIDENCE_PATH.write_text("\n".join(lines) + "\n", encoding='utf-8')
print('saved:', EVIDENCE_PATH)
if backup_path is not None:
    print('backup:', backup_path)

flow_sign_df


saved: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_evidence_summary_2026-02-23.md
backup: /Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_evidence_summary_2026-02-23.md.bak.20260223203602


subperiod  flow_t_coef  flow_t_pvalue  flow_t_minus_2_coef  \
0  2011_2026    -0.001586   1.594270e-09            -0.000655   
1  2018_2026    -0.001742   8.834517e-05            -0.000938   
2  2020_2026    -0.001809   2.661651e-03            -0.001125   

   flow_t_minus_2_pvalue  
0               0.039944  
1               0.068754  
2               0.084366

In [8]:
artifacts = sorted(RESULT_DIR.glob('*.csv'))
for p in artifacts:
    print(p)
print(EVIDENCE_PATH)

{
    'panel_rows': len(panel_out),
    'var_rows': len(var_out),
    'cirf_rows': len(cirf_out),
    'weekly_rows': len(weekly_out),
    'weekly_status': weekly_status,
}

/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/cirf_table.csv
/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/panel_coefficients.csv
/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/var_coefficients.csv
/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_results/weekly_lead_coefficients.csv
/Users/kencharoff/workspace/projects/momentum/p04_fund_flows/outputs/phaseB_evidence_summary_2026-02-23.md


{'panel_rows': 42,
 'var_rows': 54,
 'cirf_rows': 63,
 'weekly_rows': 1,
 'weekly_status': 'skipped_missing_mf'}

## Next steps

- `data/raw/mf_weekly_flows.csv` を追加すると、週次ETF vs MF回帰（Section 5.1.2近似）が自動で有効化される。
- 厳密再現へ進む場合は、`Shrout` と `NAV` の日次系列、およびフロー報告ラグ補正情報（N-CSRS/N-30b-2対応）を投入する。